# 自定义 policy 模块中的 feature_extractor 和 mlp_extractor 

In [1]:
import gymnasium as gym
import torch as th
import torch.nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

import EvnOneStock

import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# 1. 自定义特征提取器（保持你的代码）
class CustomCombinedExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 256):
        super().__init__(observation_space, features_dim)
        n_input_channels = observation_space.shape[0]
        self.layers = nn.Sequential(
            nn.Linear(n_input_channels, 128),
            nn.ReLU(),
            nn.Linear(128, features_dim),
            nn.ReLU(),
        )
    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.layers(observations)

# 2. 修改后的 policy_kwargs
policy_kwargs = dict(
    features_extractor_class=CustomCombinedExtractor,
    features_extractor_kwargs=dict(features_dim=256),
    # 重点：对于股票交易这类连续动作，使用 Squashed Gaussian 或简单的 Tanh 激活有时有帮助
    # 但 SB3 的 PPO 默认在输出层之后不加激活函数，它靠 Box 空间的 Bound 来裁剪动作。
    net_arch=dict(
        pi=[128, 64], 
        vf=[128, 64]
    ),
    # 可以选择激活函数，Tanh 在连续控制任务中通常比 ReLU 更平滑
    activation_fn=th.nn.Tanh 
)

df = pd.read_csv("ohlcv_000001.SZ.csv").fillna(0)

env = EvnOneStock.SingleStockTradingEnv(df)

# 3. 创建模型
# 注意：SB3 会根据 env.action_space 自动识别是连续动作还是离散动作
model = PPO("MlpPolicy", env, policy_kwargs=policy_kwargs, verbose=0)

model.learn(total_timesteps=10000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


D:\pyvenv\rl\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 204  |
|    iterations      | 1    |
|    time_elapsed    | 10   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 166          |
|    iterations           | 2            |
|    time_elapsed         | 24           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0043315487 |
|    clip_fraction        | 0.0485       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.42        |
|    explained_variance   | -0.0864      |
|    learning_rate        | 0.0003       |
|    loss                 | 1.22         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00494     |
|    std                  | 1            |
|    value_loss           | 2.57         |
----------------

In [14]:
model.save("PPO_v1")

In [ ]:
from stable_baselines3 import PPO

# 1. 从文件加载模型
# 注意：不需要加 .zip 后缀，SB3 会自动寻找
loaded_model = PPO.load("ppo_model_v1")

# 2. 进行推理（预测）
obs, info = env.reset()
for _ in range(1000):
    # predict 函数返回两个值：action 和 _states（对于非循环策略，_states 通常为 None）
    # deterministic=True 建议在推理阶段开启，以获得稳定的表现
    action, _states = loaded_model.predict(obs, deterministic=True)
    
    obs, reward, terminated, truncated, info = env.step(action)
    
    if terminated or truncated:
        obs, info = env.reset()

In [3]:
log_pd = pd.read_csv("EvnOneStock_20260109161047.csv")
log_pd[log_pd["date"]==log_pd["date"].max()]

,date,reward,terminated,truncated,portfolio_value,position,drawdown,current_step,prev_close,open_price,close_price,prev_position,delta_position
8448,20251224,-0.456504,False,True,8.674385e+06,0.010173,0.456199,8468,954.9845,954.1584,953.3323,0.342872,-0.3327


In [6]:
log_pd.iloc[8449:8455]

,date,reward,terminated,truncated,portfolio_value,position,drawdown,current_step,prev_close,open_price,close_price,prev_position,delta_position
8449,19910503,0.000000,False,False,1000000.000000,0.000000,0.000000,20,61.26,60.95,60.95,0.000000,0.000000
8450,19910506,0.000000,False,False,1000000.000000,0.544879,0.000000,21,60.95,60.64,60.64,0.000000,0.544879
8451,19910507,-0.010962,False,False,994518.858511,0.626260,0.005481,22,60.64,60.03,60.03,0.544879,0.081380
8452,19910508,-0.011723,False,False,991406.279467,0.246672,0.008594,23,60.03,59.73,59.73,0.626260,-0.379588
8453,19910509,-0.010979,False,False,990218.934741,0.556701,0.009781,24,59.73,59.44,59.44,0.246672,0.310029
8454,19910510,-0.015373,False,False,987436.690107,0.328578,0.012563,25,59.44,59.14,59.14,0.556701,-0.228122


In [13]:
pic_pd = log_pd[:8449].reset_index(drop=True)
pic_pd["dt"] = pd.to_datetime(pic_pd["date"], format="%Y%m%d")
pic_pd

,date,reward,terminated,truncated,portfolio_value,position,drawdown,current_step,prev_close,open_price,close_price,prev_position,delta_position,dt
0,19910503,0.000000,False,False,1.000000e+06,0.485819,0.000000,20,61.2600,60.9500,60.9500,0.000000,0.485819,1991-05-03
1,19910506,-0.004942,False,False,9.975291e+05,0.000000,0.002471,21,60.9500,60.6400,60.6400,0.485819,-0.485819,1991-05-06
2,19910507,-0.002471,False,False,9.975291e+05,0.000000,0.002471,22,60.6400,60.0300,60.0300,0.000000,0.000000,1991-05-07
3,19910508,-0.002471,False,False,9.975291e+05,1.000000,0.002471,23,60.0300,59.7300,59.7300,0.000000,1.000000,1991-05-08
4,19910509,-0.012169,False,False,9.926859e+05,1.000000,0.007314,24,59.7300,59.4400,59.4400,1.000000,0.000000,1991-05-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8444,20251218,-0.456891,False,False,8.667874e+06,0.000000,0.456607,8464,952.5062,950.8539,961.5934,0.161971,-0.161971,2025-12-18
8445,20251219,-0.456711,False,False,8.667292e+06,0.078030,0.456643,8465,961.5934,960.7673,959.9412,0.000000,0.078030,2025-12-19
8446,20251222,-0.456746,False,False,8.666715e+06,0.000000,0.456680,8466,959.9412,959.1150,951.6800,0.078030,-0.078030,2025-12-22
8447,20251223,-0.454842,False,False,8.677033e+06,0.342872,0.456033,8467,951.6800,951.6800,954.9845,0.000000,0.342872,2025-12-23


In [3]:
import torch as th
import numpy as np

# 1. 采样状态
random_obs = env.observation_space.sample()

# 2. 准备 Tensor 并确保设备一致
# model.device 会自动返回 'cuda' 或 'cpu'
obs_tensor = th.as_tensor(random_obs).float().unsqueeze(0).to(model.device)

# 3. 手动推断分布
with th.no_grad():
    # 获取动作分布
    dist = model.policy.get_distribution(obs_tensor)
    
    # 提取概率（转回 CPU 以便 numpy/print 处理）
    probs = dist.distribution.probs.cpu().numpy()[0]
    
    print(f"--- 状态测试 ---")
    print(f"输入状态: {random_obs}")
    print(f"动作概率: 向左(0): {probs[0]:.2%}, 向右(1): {probs[1]:.2%}")
    
    # 选出概率最大的动作
    action = np.argmax(probs)
    print(f"最终决策: {action}")

--- 状态测试 ---
输入状态: [-1.3300587  0.816699   0.0824696  1.2880276]
动作概率: 向左(0): 14.47%, 向右(1): 85.53%
最终决策: 1


In [4]:
random_obs

array([-1.3300587,  0.816699 ,  0.0824696,  1.2880276], dtype=float32)

In [6]:
model.policy

ActorCriticPolicy(
  (features_extractor): CustomCombinedExtractor(
    (layers): Sequential(
      (0): Linear(in_features=4, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): ReLU()
    )
  )
  (pi_features_extractor): CustomCombinedExtractor(
    (layers): Sequential(
      (0): Linear(in_features=4, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): ReLU()
    )
  )
  (vf_features_extractor): CustomCombinedExtractor(
    (layers): Sequential(
      (0): Linear(in_features=4, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): ReLU()
    )
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=256, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    

In [8]:
id(model.policy.features_extractor), id(model.policy.vf_features_extractor), id(model.policy.pi_features_extractor)

(2315767588112, 2315767588112, 2315767588112)

In [5]:
# 1. 定义自定义特征提取器 和 mlp_extractor 自定义

import torch as th
import torch.nn as nn
from stable_baselines3.common.policies import ActorCriticPolicy

# 1. 定义一个完全自定义的 MlpExtractor 类
class MyCustomMlpExtractor(nn.Module):
    def __init__(self, feature_dim: int):
        super().__init__()
        # 定义输出维度，必须告知 Policy 最终输出给 action_net 的维度是多少
        self.latent_dim_pi = 64
        self.latent_dim_vf = 64

        # 策略网络分支：加入 Dropout 层作为示例
        self.policy_net = nn.Sequential(
            nn.Linear(feature_dim, 128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(128, self.latent_dim_pi),
            nn.ReLU()
        )

        # 价值网络分支
        self.value_net = nn.Sequential(
            nn.Linear(feature_dim, 128),
            nn.Tanh(), # 甚至可以在这里用不同的激活函数
            nn.Linear(128, self.latent_dim_vf),
            nn.Tanh()
        )

    def forward(self, features: th.Tensor):
        # 返回 (policy_latent, value_latent)
        return self.policy_net(features), self.value_net(features)

    # 为了兼容性，SB3 需要这两个方法
    def forward_actor(self, features: th.Tensor) -> th.Tensor:
        return self.policy_net(features)

    def forward_critic(self, features: th.Tensor) -> th.Tensor:
        return self.value_net(features)

# 2. 定义一个新的 Policy 类来使用这个 Extractor
class CustomPolicy(ActorCriticPolicy):
    def _build_mlp_extractor(self) -> None:
        # 这里用我们自定义的类替换默认的 MlpExtractor
        self.mlp_extractor = MyCustomMlpExtractor(self.features_dim)

# 3. 使用这个自定义 Policy
model = PPO(CustomPolicy, env, policy_kwargs=policy_kwargs, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [10]:
model.policy

CustomPolicy(
  (features_extractor): CustomCombinedExtractor(
    (layers): Sequential(
      (0): Linear(in_features=4, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): ReLU()
    )
  )
  (pi_features_extractor): CustomCombinedExtractor(
    (layers): Sequential(
      (0): Linear(in_features=4, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): ReLU()
    )
  )
  (vf_features_extractor): CustomCombinedExtractor(
    (layers): Sequential(
      (0): Linear(in_features=4, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): ReLU()
    )
  )
  (mlp_extractor): MyCustomMlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
      (3): Linear(in_features=128, out_featur

In [20]:
action = np.array([0.5], dtype=np.float32)
state, reward, terminated, truncated, info = env.step(action)
assert not np.any(np.isnan(state)), "Observation contains NaN!"
assert not np.any(np.isinf(state)), "Observation contains Inf!"
assert np.isfinite(reward), "Reward is not finite!"

In [1]:
import gymnasium as gym
import torch as th
import torch.nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

import EvnOneStock

import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv("ohlcv_000001.SZ.csv")

env = EvnOneStock.SingleStockTradingEnv(df=df, lookback_n=5)


env.reset()

info_dict = {
    # "state":[],
    "reward":[],
    "terminated":[],
    "truncated":[],
    "portfolio_value":[],
    "position":[],
    "drawdown":[],
    "current_step":[],
    "prev_close":[],
    "open_price":[],
    "close_price":[],
    "prev_position":[],
    "delta_position":[],
}
    
for i in range(6):
    action = np.array([0], dtype=np.float32)
    if i%3==1:
        action = np.array([0.5], dtype=np.float32)
    elif i%3==2:
        action = np.array([1], dtype=np.float32)
    state, reward, terminated, truncated, info = env.step(action)
    info_dict["reward"].append(reward)
    info_dict["terminated"].append(terminated)
    info_dict["truncated"].append(truncated)
    for k in list(info_dict.keys())[3:]:
        info_dict[k].append(info[k])

info_pd = pd.DataFrame(info_dict)

In [16]:
info_pd

,reward,terminated,truncated,portfolio_value,position,drawdown,current_step,prev_close,open_price,close_price,prev_position,delta_position
0,0.000000,False,False,1000000.000000,0.0,0.000000,5,47.80,47.56,47.56,0.0,0.0
1,0.000000,False,False,1000000.000000,0.5,0.000000,6,47.56,47.56,47.56,0.0,0.5
2,-0.010093,False,False,994953.742641,1.0,0.005046,7,47.56,47.08,47.08,0.5,0.5
3,-0.034708,False,False,980160.462695,0.0,0.019840,8,47.08,46.38,46.38,1.0,-1.0
4,-0.019840,False,False,980160.462695,0.5,0.019840,9,46.38,46.15,46.15,0.0,0.5
5,-0.024774,False,False,977718.026006,1.0,0.022282,10,46.15,45.92,45.92,0.5,0.5


In [13]:
df.head(10)

,date,open,high,low,close,volume
0,19910403,49.00,49.00,49.00,49.00,1.0
1,19910404,48.76,48.76,48.76,48.76,3.0
2,19910405,48.52,48.52,48.52,48.52,2.0
3,19910408,48.04,48.04,48.04,48.04,2.0
4,19910409,47.80,47.80,47.80,47.80,4.0
5,19910410,47.56,47.56,47.56,47.56,15.0
6,19910411,47.56,47.56,47.56,47.56,0.0
7,19910412,47.08,47.08,47.08,47.08,8.0
8,19910416,46.38,46.38,46.38,46.38,2.0
9,19910417,46.15,46.15,46.15,46.15,1.0


In [17]:
state

array([47.56, 47.56, 47.56, 47.56,  0.  , 47.08, 47.08, 47.08, 47.08,
        8.  , 46.38, 46.38, 46.38, 46.38,  2.  , 46.15, 46.15, 46.15,
       46.15,  1.  , 45.92, 45.92, 45.92, 45.92,  4.  ,  1.  ],
      dtype=float32)

In [5]:
EvnOneStock.SingleStockTradingEnv(df=df, lookback_n=5)

In [7]:
df.loc[0:4]

,date,open,high,low,close,volume
0,19910403,49.00,49.00,49.00,49.00,1.0
1,19910404,48.76,48.76,48.76,48.76,3.0
2,19910405,48.52,48.52,48.52,48.52,2.0
3,19910408,48.04,48.04,48.04,48.04,2.0
4,19910409,47.80,47.80,47.80,47.80,4.0


In [6]:
9%2

1

In [4]:
return_pd = pd.read_csv("EvnOneStock_20260108152545.csv")
return_pd

,reward,terminated,truncated,portfolio_value,position,drawdown,current_step,prev_close,open_price,close_price,prev_position,delta_position
0,0.000000,False,False,1.000000e+06,0.062328,0.000000,20,61.2600,60.9500,60.9500,0.000000,0.062328
1,-0.000634,False,False,9.996830e+05,0.000000,0.000317,21,60.9500,60.6400,60.6400,0.062328,-0.062328
2,-0.000317,False,False,9.996830e+05,0.000000,0.000317,22,60.6400,60.0300,60.0300,0.000000,0.000000
3,-0.000317,False,False,9.996830e+05,0.054134,0.000317,23,60.0300,59.7300,59.7300,0.000000,0.054134
4,-0.000843,False,False,9.994202e+05,0.081649,0.000580,24,59.7300,59.4400,59.4400,0.054134,0.027515
...,...,...,...,...,...,...,...,...,...,...,...,...
2043,-0.164601,False,False,4.299765e+06,0.000000,0.151222,2063,297.6043,293.4619,282.0704,1.000000,-1.000000
2044,-0.151222,False,False,4.299765e+06,0.000000,0.151222,2064,282.0704,278.3164,286.4717,0.000000,0.000000
2045,-0.151222,False,False,4.299765e+06,0.000000,0.151222,2065,286.4717,286.8600,284.7888,0.000000,0.000000
2046,-0.149534,False,False,4.303691e+06,1.000000,0.150447,2066,284.7888,283.4943,283.7532,0.000000,1.000000


In [7]:
return_pd[["portfolio_value","position"]]

,portfolio_value,position
0,1.000000e+06,0.062328
1,9.996830e+05,0.000000
2,9.996830e+05,0.000000
3,9.996830e+05,0.054134
4,9.994202e+05,0.081649
...,...,...
2043,4.299765e+06,0.000000
2044,4.299765e+06,0.000000
2045,4.299765e+06,0.000000
2046,4.303691e+06,1.000000
